Traduction du texte avec Translators et google.

In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import RktnChallenge.preprocessing.translater
importlib.reload(RktnChallenge.preprocessing.translater)
from RktnChallenge.preprocessing.translater import LanguageDetector
from RktnChallenge.preprocessing.translater import Translator

import RktnChallenge.preprocessing.CopyWhereNan
importlib.reload(RktnChallenge.preprocessing.CopyWhereNan)
from RktnChallenge.preprocessing.CopyWhereNan import CopyWhereNan

import RktnChallenge.preprocessing.removeHTML
importlib.reload(RktnChallenge.preprocessing.removeHTML)
from RktnChallenge.preprocessing.removeHTML import removeHTML

2023-10-02 10:29:21.729806: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 10:29:21.755315: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9511] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 10:29:21.755340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 10:29:21.755356: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 10:29:21.760516: I tensorflow/core/platform/cpu_feature_guar

On charge la base et init le support GPU / CPU

In [2]:
RktnModel = ModelTrainer("data.csv")

# On initialise le GPU disponible, la gestion mémoire
#RktnModel.initGPU()

On détecte la langue de la source et range dans une nouvelle colonne

In [ ]:
desc_removeHTML = removeHTML("description")
desi_removeHTML = removeHTML("designation")

desi_langueDetect = LanguageDetector("desi_langue","designation",confidence=0.8)
desc_langueDetect = LanguageDetector("desc_langue","description",confidence=0.8)


desi_Translator = Translator(dest = "tr_designation", src="designation",               
                             detected_lang="desi_langue", source = "reverso",verbose = True)

desc_Translator = Translator(dest = "tr_description", src="description",               
                             detected_lang="desc_langue", source = "reverso", verbose = True)



RktnModel.initPreprocess()
# unidecode et removeHTML
RktnModel.addPreprocessStep("desc removeHTML",desc_removeHTML)
RktnModel.addPreprocessStep("desi removeHTML",desi_removeHTML)
# decode la langue et traduit la designation
RktnModel.addPreprocessStep("detect langue designation",desi_langueDetect)
RktnModel.addPreprocessStep("translate designation",desi_Translator)
# decode la langue et traduit la description
RktnModel.addPreprocessStep("detect langue description",desc_langueDetect)
RktnModel.addPreprocessStep("translate description",desc_Translator)
RktnModel.preprocess()


print("langues detectees dans les designations: ",desi_langueDetect.langues)
print("langues detectees dans les descriptions: ",desc_langueDetect.langues)



On vient de traduire dans tr_ ce qui n'est pas en francais, on complete avec le français:

In [ ]:
# on copie les lignes qui n'ont pas été traduites (celles en français)
desc_copy = CopyWhereNan("tr_description","description")
desi_copy = CopyWhereNan("tr_designation","designation")

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("desc copy french",desc_copy)
RktnModel.addPreprocessStep("desi copy french",desi_copy)
RktnModel.data = RktnModel.preprocess()


#on sauve
RktnModel.saveCSV("data_tr_08_.csv")
